In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# New section

In [ ]:
import pandas as pd
import os
os.chdir('/content/gdrive/MyDrive/Colab Notebooks')
#!ls
print("Loading Data")
labels = pd.read_csv('Labels.csv', header=None)
Training = pd.read_csv('Training_Data.csv', header=None)
# Stack real and imaginary components in the desired format
num_samples = Training.shape[0]  # Get the number of samples
num_features = Training.shape[1]  # Get the number of original features

Loading Data


In [ ]:
print("Loading Dependencies")
from __future__ import division, print_function, absolute_import
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
 #1. Load Data
# 2. Preprocess Data
Training_complex = Training.applymap(lambda x: complex(str(x).replace('i', 'j')))
real_df = Training_complex.applymap(lambda x: x.real)
imag_df = Training_complex.applymap(lambda x: x.imag)
Stacked_Training_input = np.zeros((4096, 839, 2, 1), dtype=np.float32)

#Stacked_Training_input = pd.concat([real_df, imag_df], axis=1)  # Stack in columns first
#Stacked_Training_input = Stacked_Training_input.values.reshape(num_samples, num_features, 2)  # Reshape to (samples, features, 2)
#Stacked_Training_input = Stacked_Training_input.transpose(0, 2, 1)  # Transpose to (samples, 2, features)
#Stacked_Training_input = Stacked_Training_input.reshape(num_samples, num_features * 2, 1, 1)  # Reshape to final format
Stacked_Training_input[:, :, 0, 0] = real_df.values  # Real component in channel 0
Stacked_Training_input[:, :, 1, 0] = imag_df.values  # Real component in channel 0

labels = labels.T  # Transpose to get (4096, 128)

Loading Dependencies


<ipython-input-55-ca041ebf4ae8>:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  Training_complex = Training.applymap(lambda x: complex(str(x).replace('i', 'j')))
<ipython-input-55-ca041ebf4ae8>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  real_df = Training_complex.applymap(lambda x: x.real)
<ipython-input-55-ca041ebf4ae8>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  imag_df = Training_complex.applymap(lambda x: x.imag)


In [ ]:
# 3. Define the Neural Network
class ReshapeLayer(tf.keras.layers.Layer):
    def __init__(self, target_shape, **kwargs):
        super(ReshapeLayer, self).__init__(**kwargs)
        self.target_shape = target_shape

    def call(self, inputs):
        return tf.reshape(inputs, self.target_shape)

    def get_config(self):
        config = super(ReshapeLayer, self).get_config()
        config.update({'target_shape': self.target_shape})
        return config

def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    with tf.compat.v1.variable_scope('ConvNet', reuse=reuse):
        x = x_dict['images']
#        x = tf.expand_dims(tf.expand_dims(x, axis=-1), axis=1)  # Reshape to (None, 1, 839, 2, 1)
        x = ReshapeLayer(target_shape=(-1, 1, 839, 2, 1))(x)


        # More convolutional layers with increasing filters
        conv1 = tf.keras.layers.Conv3D(32, [1, 3, 2], activation='relu', padding='same')(x)
        conv1 = tf.keras.layers.MaxPooling3D([1, 2, 1], [1, 2, 1], padding='same')(conv1)
        conv2 = tf.keras.layers.Conv3D(64, [1, 3, 2], activation='relu', padding='same')(conv1)
        conv2 = tf.keras.layers.MaxPooling3D([1, 2, 1], [1, 2, 1], padding='same')(conv2)
        conv3 = tf.keras.layers.Conv3D(128, [1, 3, 2], activation='relu', padding='same')(conv2)  # Added layer
        conv3 = tf.keras.layers.MaxPooling3D([1, 2, 1], [1, 2, 1], padding='same')(conv3)  # Added layer

        # Flatten and dense layers
        fc1 = tf.keras.layers.Flatten()(conv3)
        fc1 = tf.keras.layers.Dense(2048, activation='relu')(fc1)  # Increased units, added activation
        fc1 = tf.keras.layers.Dropout(rate=dropout)(fc1)
        fc2 = tf.keras.layers.Dense(1024, activation='relu')(fc1)  # Added another dense layer
        fc2 = tf.keras.layers.Dropout(rate=dropout)(fc2)

        # Output layer
        out = tf.keras.layers.Dense(n_classes)(fc2)
    return out

# 4. Create and Compile the Model
X_input = tf.keras.Input(shape=(839, 2, 1), dtype=tf.float32)  # Updated input shape
learning_rate = 0.001
num_steps = 2000
batch_size = 10
num_classes = 128
logits = conv_net({'images': X_input}, num_classes, 0.25, reuse=False, is_training=True)
model = tf.keras.Model(inputs=X_input, outputs=logits)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

# 5. Train the Model
model.fit(Stacked_Training_input, labels,
          batch_size=batch_size, epochs=num_steps // (len(Stacked_Training_input) // batch_size),
          verbose=1)

# 6. Evaluate the Model (Add your evaluation code here)

Epoch 1/4
410/410 ━━━━━━━━━━━━━━━━━━━━ 477s 1s/step - binary_accuracy: 0.9797 - loss: 0.2992
Epoch 2/4
410/410 ━━━━━━━━━━━━━━━━━━━━ 501s 1s/step - binary_accuracy: 0.9844 - loss: 0.2518
Epoch 3/4
410/410 ━━━━━━━━━━━━━━━━━━━━ 468s 1s/step - binary_accuracy: 0.9844 - loss: 0.2518
Epoch 4/4
410/410 ━━━━━━━━━━━━━━━━━━━━ 506s 1s/step - binary_accuracy: 0.9844 - loss: 0.2518


In [ ]:
print(Stacked_Training_input[0,0,1,:])


[-0.]
